In [1]:
import pandas as pd
import numpy as np
import torchvision as tv
import jieba
from gensim import models
from gensim.models import Word2Vec
import torchvision.transforms as transforms
from torchvision.transforms import ToPILImage
import torch
show = ToPILImage() # 可以把Tensor转成Image，方便可视化
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('../../data/df_sen_sub/train.csv')
data

,content_id,content,subject,sentiment_value,sentiment_word
0,vUXizsqexyZVRdFH,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,价格,0,影响
1,4QroPd9hNfnCHVt7,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,价格,-1,高
2,QmqJ2AvM5GplaRyz,斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...,价格,1,低
3,KMT1gFJiU4NWrVDn,这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了,价格,-1,有钱任性
4,nVIlGd5yMmc37t1o,17价格忒高，估计也就是14-15左右。,价格,-1,高
5,TVciHBPL5XmUxMEd,我开始就是荣放2.5 森林人2.5二选一 荣放主要是底盘质感不行 太硬 其次是...,价格,1,便宜
6,4lb9TsO5rjqUy1Bu,唉，这货的价格死硬死硬的，低配版优惠1万据说已经罕有了。,价格,-1,死硬
7,z98xV7MphZIF4EvO,价格的话只能说一般般吧，太仓前段时间定的比你便宜！,价格,0,一般
8,hoWrMnGTUvdsK3Dm,听过，价格太贵，但一直念念不忘,价格,-1,贵
9,r1WHgbI0ZKqMdELy,恭喜恭喜，这个优惠不错哦！,价格,1,优惠不错


In [3]:
'''
data = pd.read_csv('../../data/df_sen_sub/train.csv')
len(data['content_id'].unique()), len(list(data['content_id']))
content_ids = data['content_id'].unique()
subjs = data['subject'].unique()
df = {}
text, subs, sentis = [], [], []
for i in range(len(content_ids)):
    if i % 100 == 0:print(i)
    dd = data[data['content_id'] == content_ids[i]]
    text.append(dd.loc[dd.index.tolist()[0], 'content'])
    temp_subs, temp_senti = [], []
    for sub in subjs:
        temp_subs.append(sub)
        if len(dd[dd['subject'] == sub]) != 0:
            temp_senti.append(str(int(dd[dd['subject'] == sub]['sentiment_value'])))
        else:
            temp_senti.append('2')
    subs.append(','.join(temp_subs))
    sentis.append(','.join(temp_senti))
df['content'] = text
df['subjects'] = subs
df['sentiments'] = sentis
df = pd.DataFrame(df)
df.to_csv('../../data/df_sen_sub/merge_train.csv', index = False)
#data = pd.read_csv('../../data/df_sen_sub/test_public.csv')
'''

"\ndata = pd.read_csv('../../data/df_sen_sub/train.csv')\nlen(data['content_id'].unique()), len(list(data['content_id']))\ncontent_ids = data['content_id'].unique()\nsubjs = data['subject'].unique()\ndf = {}\ntext, subs, sentis = [], [], []\nfor i in range(len(content_ids)):\n    if i % 100 == 0:print(i)\n    dd = data[data['content_id'] == content_ids[i]]\n    text.append(dd.loc[dd.index.tolist()[0], 'content'])\n    temp_subs, temp_senti = [], []\n    for sub in subjs:\n        temp_subs.append(sub)\n        if len(dd[dd['subject'] == sub]) != 0:\n            temp_senti.append(str(int(dd[dd['subject'] == sub]['sentiment_value'])))\n        else:\n            temp_senti.append('2')\n    subs.append(','.join(temp_subs))\n    sentis.append(','.join(temp_senti))\ndf['content'] = text\ndf['subjects'] = subs\ndf['sentiments'] = sentis\ndf = pd.DataFrame(df)\ndf.to_csv('../../data/df_sen_sub/merge_train.csv', index = False)\n#data = pd.read_csv('../../data/df_sen_sub/test_public.csv')\n"

In [4]:
data = pd.read_csv('../../data/df_sen_sub/merge_train.csv')
data = data.fillna('')
subs = list(data['subjects'])
sentiments = list(data['sentiments'])
content = list(data['content'])
test_data = pd.read_csv('../../data/df_sen_sub/test_public.csv')
test_data = test_data.fillna('')
test_content = list(test_data['content'])
data = pd.read_csv('../../data/df_sen_sub/train.csv')
data = data.fillna('')
senti_words = list(data['sentiment_word'])

In [5]:
with open('../../data/df_sen_sub/add_text.txt','w') as f:
    f.write('\n'.join(list(content) + list(test_content)  + list(subs) + senti_words))
# 迭代器，使用jieba将句子进行分词
stopwords = open('stopwords.txt', 'r').read().split('\n')
class Sentences(object):# 这个类可以根据实际情况重写，我已经将所有的文章进行分句，并整合到了一个文件里面
    def __init__(self, dirname):
        self.dirname = dirname # 句子所在文件，没句句子占一行
        #jieba.load_userdict("wordBase.txt") # 加载词库

    def __iter__(self):
        #for fname in os.listdir(self.dirname):
        for line in open(self.dirname):
                line = line.replace('\n', '')
                res = []
                for word in list(jieba.cut(line)):
                    if word in stopwords:
                        continue
                    res.append(word)
                yield res#list(jieba.cut(line))

sentences = []
def train_word2vec(folder_path, size=100):
    global sentences
    sentences = Sentences(folder_path) #生成分词后的句子，是一个二维数组

    # size是词向量长度
    # worker是线程数量，建议与物理线程数量一致
    # min_count是指出现次数小于一定程度，就忽略，0表示不忽略
    #model = Word2Vec(sentences, size=size, workers=8, min_count=0)

    # 训练结束就将模型保存起来
    #model.save("../../data/df_sen_sub/add_word2vec_model")

# 生成50维度的词向量模型
train_word2vec("../../data/df_sen_sub/add_text.txt",100)

# 测试训练好的词向量模型，使用model[keyWord]即可获取keyword这个词的词向量
model = Word2Vec.load("../../data/df_sen_sub/add_word2vec_model")
sentences = list(sentences)


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.962 seconds.
Prefix dict has been built succesfully.


In [6]:
len(sentences)

28890

In [7]:
from collections import Counter
data = pd.read_csv('../../data/df_sen_sub/train.csv')
sub_uniq = data['subject'].unique()
data = data[pd.notnull(data['sentiment_word'])]
senti_word_set = {}
sum_ = 0
for key in sub_uniq:
    co = 0
    senti_word_set[key] = {}
    #dd = list(data[data['subject'] == key]['sentiment_word'])
    ddd = data[data['subject'] == key]
    for kk in [-1, 0, 1]:
        dd = list(ddd[ddd['sentiment_value'] == kk]['sentiment_word'])
        ss = []
        for phras in dd:
            words = list(jieba.cut(phras))
            for word in words:
                word = word.replace(' ', '')
                if word == '' or word in stopwords:continue
                ss.append(word)
                co += 1
            counter = Counter(ss)
            sort = sorted(counter.items(), key=lambda item: item[1], reverse = True)
            for mm in counter:
                if mm not in senti_word_set[key]:
                    senti_word_set[key][mm] = kk
                elif senti_word_set[key][mm] == kk:
                    continue
                else:
                    senti_word_set[key][mm] = 0
        sum_ += co
print(sum_)
senti_word_set

6431


{'价格': {'高': 0,
  '有钱': -1,
  '任性': -1,
  '死硬': -1,
  '贵': 0,
  '优惠': 0,
  '太小': -1,
  '套路': -1,
  '太贵': -1,
  '不值': -1,
  '高昂': -1,
  '下不来': -1,
  '不会': 0,
  '便宜': 0,
  '放弃': -1,
  '平易近人': -1,
  '价格': 0,
  '老贵': -1,
  '没有': 0,
  '保证': -1,
  '好多': -1,
  '更': -1,
  '合适': 0,
  '那点': -1,
  '贵太多': -1,
  '挺高': -1,
  '不菲': -1,
  '一点': 0,
  '奇高': -1,
  '不好': -1,
  '小贵': 0,
  '过高': -1,
  '售价': -1,
  '明显': -1,
  '高于': -1,
  '竞品': -1,
  '优势': -1,
  '虚高': -1,
  '不高': 0,
  '性价比': 0,
  '不算': -1,
  '单价': -1,
  '没': 0,
  '降价': -1,
  '不要': 0,
  '想': -1,
  '保值': -1,
  '率': -1,
  '大众': -1,
  '好': 0,
  '略贵': -1,
  '保养': -1,
  '不用': -1,
  '考虑': -1,
  '不靠': -1,
  '谱': -1,
  '没法': -1,
  '咬咬牙': -1,
  '不合算': -1,
  '影响': 0,
  '亲民点': 0,
  '差不多': 0,
  '差距': 0,
  '知道': 0,
  '合理': 0,
  '就行': 0,
  '行情': 0,
  '价': 0,
  '正常': 0,
  '几百': 0,
  '参差不齐': 0,
  '行': 0,
  '买': 0,
  '价差': 0,
  '很大': 0,
  '低': 0,
  '不能': 0,
  '在意': 0,
  '还行': 0,
  '不错': 1,
  '幅度': 1,
  '不小': 1,
  '后悔': 1,
  '值': 1,
  '愉快': 1,
  '不够': 1,
  '比较'

In [8]:
x_vecs = []
x_test_vecs = []  
ind = 0
word2ind = {}
pretrained = []
senti_sen = []
senti_test_sen = []
for i in range(len(test_content) + len(content)):
    if i < len(content):
        temp = []
        t_s = []
        for j in range(len(sentences[i])):
            if sentences[i][j] not in word2ind:
                pretrained.append(model[sentences[i][j]])
                word2ind[sentences[i][j]] = ind
                ind += 1
            t_s.append(sentences[i][j])
            temp.append(model[sentences[i][j]])
        while len(temp) < 50:
            temp.append([0.0] * 100)
        if len(temp) > 50:
            temp = temp[:50]
        x_vecs.append(temp)
        senti_sen.append(t_s[:50])
    else:
        temp = []
        t_s = []
        for j in range(len(sentences[i])):
            if sentences[i][j] not in word2ind:
                pretrained.append(model[sentences[i][j]])
                word2ind[sentences[i][j]] = ind
                ind += 1
            t_s.append(sentences[i][j])
            temp.append(model[sentences[i][j]])
        while len(temp) < 50:
            temp.append([0.0] * 100)
        if len(temp) > 50:
            temp = temp[:50]
        x_test_vecs.append(temp)
        senti_test_sen.append(t_s[:50])     
x_vecs = np.array(x_vecs)
x_test_vecs = np.array(x_test_vecs)
print(x_vecs.shape, x_test_vecs.shape)

/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:34: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(8290, 50, 100) (2364, 50, 100)


In [9]:
senti_word_set['价格']

{'高': 0,
 '有钱': -1,
 '任性': -1,
 '死硬': -1,
 '贵': 0,
 '优惠': 0,
 '太小': -1,
 '套路': -1,
 '太贵': -1,
 '不值': -1,
 '高昂': -1,
 '下不来': -1,
 '不会': 0,
 '便宜': 0,
 '放弃': -1,
 '平易近人': -1,
 '价格': 0,
 '老贵': -1,
 '没有': 0,
 '保证': -1,
 '好多': -1,
 '更': -1,
 '合适': 0,
 '那点': -1,
 '贵太多': -1,
 '挺高': -1,
 '不菲': -1,
 '一点': 0,
 '奇高': -1,
 '不好': -1,
 '小贵': 0,
 '过高': -1,
 '售价': -1,
 '明显': -1,
 '高于': -1,
 '竞品': -1,
 '优势': -1,
 '虚高': -1,
 '不高': 0,
 '性价比': 0,
 '不算': -1,
 '单价': -1,
 '没': 0,
 '降价': -1,
 '不要': 0,
 '想': -1,
 '保值': -1,
 '率': -1,
 '大众': -1,
 '好': 0,
 '略贵': -1,
 '保养': -1,
 '不用': -1,
 '考虑': -1,
 '不靠': -1,
 '谱': -1,
 '没法': -1,
 '咬咬牙': -1,
 '不合算': -1,
 '影响': 0,
 '亲民点': 0,
 '差不多': 0,
 '差距': 0,
 '知道': 0,
 '合理': 0,
 '就行': 0,
 '行情': 0,
 '价': 0,
 '正常': 0,
 '几百': 0,
 '参差不齐': 0,
 '行': 0,
 '买': 0,
 '价差': 0,
 '很大': 0,
 '低': 0,
 '不能': 0,
 '在意': 0,
 '还行': 0,
 '不错': 1,
 '幅度': 1,
 '不小': 1,
 '后悔': 1,
 '值': 1,
 '愉快': 1,
 '不够': 1,
 '比较': 1,
 '下降': 1,
 '很多': 1,
 '超级': 1,
 '最好': 1,
 '操控': 1,
 '满意': 1,
 '费用': 1,
 '相当': 1,
 '实惠': 1

In [10]:
x_sent_vecs = {}
x_test_sent_vecs = {}
for key in senti_word_set:
    x_sent_vecs[key] = []
    x_test_sent_vecs[key] = []
    for i in range(len(content) + len(test_content)):
        if i < len(content):
            t_sent = []
            for j in range(len(sentences[i])):
                if sentences[i][j] in senti_word_set[key]:
                    t_sent.append(senti_word_set[key][sentences[i][j]])
                else:
                    t_sent.append(0) 
            while len(t_sent) < 50:
                t_sent.append(0)
            if len(t_sent) > 50:
                t_sent = t_sent[:50]
            x_sent_vecs[key].append(t_sent)
        else:
            t_sent = []
            for j in range(len(sentences[i])):
                if sentences[i][j] in senti_word_set[key]:
                    t_sent.append(senti_word_set[key][sentences[i][j]])
                else:
                    t_sent.append(0) 
            while len(t_sent) < 50:
                t_sent.append(0)
            if len(t_sent) > 50:
                t_sent = t_sent[:50]
            x_test_sent_vecs[key].append(t_sent)    

In [11]:
for key in x_test_sent_vecs:
    print(key,len(x_sent_vecs['价格']))

价格 8290
配置 8290
操控 8290
舒适性 8290
油耗 8290
动力 8290
内饰 8290
安全性 8290
空间 8290
外观 8290


In [12]:
#va, content_vecs, y
pretrained.append([0.0] * 100)
word2ind['null'] = 17387
map_ = ['-1', '0', '1', '2']
sub_set = {}
for i in range(len(subs)):
    subjs = subs[i].split(',')
    sentis = sentiments[i].split(',')
    for j in range(len(subjs)):
        if subjs[j] not in sub_set:
            sub_set[subjs[j]] = []
        sub_set[subjs[j]].append(map_.index(sentis[j]))

In [13]:
pretrained = np.array(pretrained)
han_x = []
for i in range(len(senti_sen)):
    temp = []
    for j in range(len(senti_sen[i])):
        temp.append(word2ind[senti_sen[i][j]])
    while len(temp) < 50:
        temp.append(17387)

    temp = temp[:50]

    han_x.append(temp)
han_x = np.array(han_x)

han_x_test = []
for i in range(len(senti_test_sen)):
    temp = []
    for j in range(len(senti_test_sen[i])):
        temp.append(word2ind[senti_test_sen[i][j]])
    while len(temp) < 50:
        temp.append(17387)

    temp = temp[:50]

    han_x_test.append(temp)
han_x = np.array(han_x)
han_x_test = np.array(han_x_test)

In [41]:

'''
class Net(nn.Module):
    def __init__(self, pretrained, vocab_size):
        super(Net, self).__init__()
        self.bilstm = nn.LSTM(input_size = 100, hidden_size = 100, bidirectional = True) 
        #self.fc1   = nn.Linear(3750, 50) 
        #self.fc2   = nn.Linear(200, 50)
        #self.fc3   = nn.Linear(200, 50)
        #self.fc4   = nn.Linear(100, 3)
        self.fctest   = nn.Linear(200, 3)
        self.embeddings = nn.Embedding(vocab_size, 100)
        pretrained_weight = np.array(pretrained)
        self.embeddings.weight.data.copy_(torch.from_numpy(pretrained_weight))
        
    def forward(self, x, aspects): 
        x = self.embeddings(x)#batch,50,100
        #va = self.embeddings(aspects[0])
        #t_va = va.unsqueeze(0).expand(x.shape[0], x.shape[1], 100)
        output, (hn, cn) =  self.bilstm(x.float())#batch,50,200
        H, hN = output, output[:,-1,:]
        #con = torch.cat((t_va, H), 2)
        #t1 = F.max_pool2d(F.relu(con), (1, 4))
        #con = t1.view(x.size()[0], -1)
        #con = F.relu(self.fc1(con))
        #alpha = con.unsqueeze(1)
        #rr = torch.matmul(alpha,H).squeeze(1)
        #yy = torch.cat((F.relu(self.fc2(rr)), F.relu(self.fc3(hN))), 1)
        #x = self.fc4(yy)
        x  = self.fctest(hN)
        return x
    

net = Net(pretrained, len(pretrained))
print(net)
from torch import optim
criterion = nn.CrossEntropyLoss() # 交叉熵损失函数
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)
'''
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, pretrained, vocab_size):
        super(Net, self).__init__()
        #self.rnn1 = nn.RNN(100, 100) 
        self.bilstm = nn.LSTM(input_size = 100, hidden_size = 100, bidirectional = True)
        #self.fc1   = nn.Linear(1250, 120) 
        self.fch = nn.Linear(200, 100)
        self.fcv = nn.Linear(200, 100)
        self.fcw = nn.Linear(200, 1)
        self.fcwp = nn.Linear(200,50)
        self.fcwh = nn.Linear(200,50)
        self.fcwsen = nn.Linear(200, 50)
        self.fc1   = nn.Linear(3750, 120) 
        self.fc2   = nn.Linear(120, 50)
        self.fc3   = nn.Linear(50, 10)
        self.fcnn = nn.Linear(50, 10)
        self.final = nn.Linear(10, 4)
        self.cnn = nn.Conv2d(
                in_channels=1,      # input height
                out_channels=50,    # n_filters
                kernel_size=(3,100),      # filter size
            )
        self.pooling = nn.MaxPool2d((1, 48))
        self.embeddings = nn.Embedding(vocab_size, 100)
        pretrained_weight = np.array(pretrained)
        self.embeddings.weight.data.copy_(torch.from_numpy(pretrained_weight))
        
    def forward(self, x, x_sent, aspects): 
        x = self.embeddings(x) #[4,50,100]
        batch_size = x.shape[0]
        N = x.shape[1]
        output, (hn, cn) =  self.bilstm(x.float())  #[4,50,200]
        xx = x.reshape((-1, 1, 50, 100))  #[4, 1, 50 ,100]
        xx = self.cnn(xx)    #[4, 50, 48, 1]
        xx = xx.reshape((x.shape[0], 1, 50, 48))
        cnn_result = self.pooling(xx).reshape((x.shape[0], 50))
        hN = output[:,-1,:]
        va = self.embeddings(aspects[0]) #[1, 100]
        t_va = torch.matmul(torch.ones((x.shape[0],50, 1)), va)#[4, 50, 100]
        x_sent = x_sent.float()
        #[4, 50] *? = [4, 50, 100]
        sent_va = torch.mul(torch.matmul(x_sent.reshape((x.shape[0], 50, 1)), torch.ones((x.shape[0],1, 100))), t_va)  #[4, 50, 100]
        t_va = torch.cat((sent_va, t_va),  2)
        t_va = t_va.reshape((-1, 200)) #[4*50, 100]
        x = output.reshape((-1, 200)) #[4*50, 200]
        x = torch.tanh(torch.cat((self.fch(x), self.fcv(t_va)), 1))
        M = x.reshape((batch_size, N, -1)) #[4, 50, 200]
        alpha = F.softmax(self.fcw(M)).reshape((batch_size, 1, -1))#[4,50]
        rr = torch.matmul(alpha, output).reshape((batch_size, -1)) #[4, 200]
        x_sent = x_sent.reshape((batch_size, 1, -1))
        ss = torch.matmul(x_sent.float(), output).reshape((batch_size, -1))
        final = torch.tanh(self.fcwh(hN) + self.fcwp(rr) + self.fcwsen(ss))
        x = self.fc3(final) 
        cnn = self.fcnn(cnn_result)
        x = self.final(torch.tanh(x + cnn))
        return x, alpha
    


In [54]:
ii = 0
import time
for key in sub_set:
    if ii == 10:
        break
    ii += 1
    net = Net(pretrained, len(pretrained))
    cur_max_acc = 0.0
    print(net)
    from torch import optim
    criterion = nn.CrossEntropyLoss() # 交叉熵损失函数
    optimizer = optim.Adagrad(net.parameters(), lr=0.01)#, momentum=0.9)
    train_x, test_x, train_y, test_y = train_test_split(han_x,sub_set[key], test_size=0.1, random_state=42)
    train_x_sents, test_x_sents, train_y_, test_y_ = train_test_split(np.array(x_sent_vecs[key]),sub_set[key], test_size=0.1, random_state=42)
    trainset = [((train_x[i],train_x_sents[i]), train_y[i]) for i in range(len(train_x))]
    testset = [((test_x[i],test_x_sents[i]), test_y[i]) for i in range(len(test_x))]
    trainloader = torch.utils.data.DataLoader(
                        trainset, 
                        batch_size=4,
                        shuffle=True, 
                        num_workers=2)
    testloader = torch.utils.data.DataLoader(
                        testset, 
                        batch_size=4,
                        shuffle=False, 
                        num_workers=2)
    torch.set_num_threads(8)
    for epoch in range(50):  

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):

            # 输入数据
            (inputs, sents), labels = data
            # 梯度清零
            optimizer.zero_grad()
            # forward + backward 
            va = [[word2ind[key]]]
            va = torch.from_numpy(np.array(va))
            outputs,alpha = net(inputs, sents, va)
            #plt.imshow(alpha.detach().numpy()[0])
            
            #plt.colorbar()
            #plt.show()  
            loss = criterion(outputs, labels)
            loss.backward()   

            # 更新参数 
            optimizer.step()

            # 打印log信息
            # loss 是一个scalar,需要使用loss.item()来获取数值，不能使用loss[0]
            running_loss += loss.item()
            if i % 200 == 199: # 每200个batch打印一下训练状态
                print('[%d, %5d] loss: %.3f' \
                      % (epoch+1, i+1, running_loss / 200))
                running_loss = 0.0
                
        with torch.no_grad():
            correct, total = 0, 0
            for data in testloader:
                (images, sentis), labels = data
                va = [[word2ind[key]]]
                va = torch.from_numpy(np.array(va))
                outputs, alpha = net(images,sentis, va)
                _, predicted = torch.max(outputs, 1)
                #import ipdb;ipdb.set_trace()
                #res = res + [map_[int(predicted[i])] for i in range(len(predicted))]
                total += labels.size(0)
                correct += (predicted == labels).sum()
            if cur_max_acc < float(100 * correct / total):
                print('-------------cur max is ::::----------'+str(float(100 * correct / total)))
                cur_max_acc = float(100 * correct / total)
                torch.save(net, 'attention++/net' + key + '.pkl')  # 保存整个网络
            print('result of epoch ' + str(epoch) + 'accuracy is : %d %%' % (100 * correct / total))

    print('key' + key)
    #net2 = torch.load('net.pkl')
    #prediction = net2(x)
    print('Finished Training')


Net(
  (bilstm): LSTM(100, 100, bidirectional=True)
  (fch): Linear(in_features=200, out_features=100, bias=True)
  (fcv): Linear(in_features=200, out_features=100, bias=True)
  (fcw): Linear(in_features=200, out_features=1, bias=True)
  (fcwp): Linear(in_features=200, out_features=50, bias=True)
  (fcwh): Linear(in_features=200, out_features=50, bias=True)
  (fcwsen): Linear(in_features=200, out_features=50, bias=True)
  (fc1): Linear(in_features=3750, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=10, bias=True)
  (fcnn): Linear(in_features=50, out_features=10, bias=True)
  (final): Linear(in_features=10, out_features=4, bias=True)
  (cnn): Conv2d(1, 50, kernel_size=(3, 100), stride=(1, 1))
  (pooling): MaxPool2d(kernel_size=(1, 48), stride=(1, 48), padding=0, dilation=1, ceil_mode=False)
  (embeddings): Embedding(19307, 100)
)


/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:90: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[1,   200] loss: 0.531
[1,   400] loss: 0.440
[1,   600] loss: 0.388
[1,   800] loss: 0.309
[1,  1000] loss: 0.271
[1,  1200] loss: 0.286
[1,  1400] loss: 0.229
[1,  1600] loss: 0.222
[1,  1800] loss: 0.255
-------------cur max is ::::----------94.0
result of epoch 0accuracy is : 94 %


/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


[2,   200] loss: 0.248
[2,   400] loss: 0.229
[2,   600] loss: 0.214
[2,   800] loss: 0.195
[2,  1000] loss: 0.202
[2,  1200] loss: 0.199
[2,  1400] loss: 0.231
[2,  1600] loss: 0.229
[2,  1800] loss: 0.173
result of epoch 1accuracy is : 94 %
[3,   200] loss: 0.193
[3,   400] loss: 0.206
[3,   600] loss: 0.155
[3,   800] loss: 0.248
[3,  1000] loss: 0.167
[3,  1200] loss: 0.184
[3,  1400] loss: 0.161
[3,  1600] loss: 0.177
[3,  1800] loss: 0.185
-------------cur max is ::::----------95.0
result of epoch 2accuracy is : 95 %
[4,   200] loss: 0.166
[4,   400] loss: 0.151
[4,   600] loss: 0.174
[4,   800] loss: 0.161
[4,  1000] loss: 0.147
[4,  1200] loss: 0.173
[4,  1400] loss: 0.208
[4,  1600] loss: 0.158
[4,  1800] loss: 0.164
result of epoch 3accuracy is : 94 %
[5,   200] loss: 0.154
[5,   400] loss: 0.180
[5,   600] loss: 0.142
[5,   800] loss: 0.131
[5,  1000] loss: 0.192
[5,  1200] loss: 0.130
[5,  1400] loss: 0.134
[5,  1600] loss: 0.159
[5,  1800] loss: 0.124
result of epoch 4accu

...

[1,   200] loss: 0.343
[1,   400] loss: 0.292
[1,   600] loss: 0.267
[1,   800] loss: 0.244
[1,  1000] loss: 0.213
[1,  1200] loss: 0.229
[1,  1400] loss: 0.223
[1,  1600] loss: 0.240
[1,  1800] loss: 0.202
-------------cur max is ::::----------93.0
result of epoch 0accuracy is : 93 %
[2,   200] loss: 0.197
[2,   400] loss: 0.175
[2,   600] loss: 0.161
[2,   800] loss: 0.162
[2,  1000] loss: 0.149
[2,  1200] loss: 0.155
[2,  1400] loss: 0.140
[2,  1600] loss: 0.138
[2,  1800] loss: 0.151
-------------cur max is ::::----------94.0
result of epoch 1accuracy is : 94 %
[3,   200] loss: 0.124
[3,   400] loss: 0.124
[3,   600] loss: 0.168
[3,   800] loss: 0.134
[3,  1000] loss: 0.134
[3,  1200] loss: 0.135
[3,  1400] loss: 0.154
[3,  1600] loss: 0.117
[3,  1800] loss: 0.106
result of epoch 2accuracy is : 94 %
[4,   200] loss: 0.111
[4,   400] loss: 0.116
[4,   600] loss: 0.113
[4,   800] loss: 0.121
[4,  1000] loss: 0.112
[4,  1200] loss: 0.138
[4,  1400] loss: 0.130
[4,  1600] loss: 0.115
[

[33,   800] loss: 0.020
[33,  1000] loss: 0.013
[33,  1200] loss: 0.014
[33,  1400] loss: 0.016
[33,  1600] loss: 0.017
[33,  1800] loss: 0.017
result of epoch 32accuracy is : 93 %
[34,   200] loss: 0.018
[34,   400] loss: 0.016
[34,   600] loss: 0.010
[34,   800] loss: 0.013
[34,  1000] loss: 0.016
[34,  1200] loss: 0.014
[34,  1400] loss: 0.018
[34,  1600] loss: 0.014
[34,  1800] loss: 0.012
result of epoch 33accuracy is : 93 %
[35,   200] loss: 0.017
[35,   400] loss: 0.014
[35,   600] loss: 0.015
[35,   800] loss: 0.013
[35,  1000] loss: 0.013
[35,  1200] loss: 0.015
[35,  1400] loss: 0.009
[35,  1600] loss: 0.014
[35,  1800] loss: 0.014
result of epoch 34accuracy is : 93 %
[36,   200] loss: 0.011
[36,   400] loss: 0.012
[36,   600] loss: 0.015
[36,   800] loss: 0.017
[36,  1000] loss: 0.014
[36,  1200] loss: 0.011
[36,  1400] loss: 0.012
[36,  1600] loss: 0.012
[36,  1800] loss: 0.011
result of epoch 35accuracy is : 93 %
[37,   200] loss: 0.011
[37,   400] loss: 0.017
[37,   600] 

[12,   200] loss: 0.085
[12,   400] loss: 0.072
[12,   600] loss: 0.061
[12,   800] loss: 0.064
[12,  1000] loss: 0.067
[12,  1200] loss: 0.055
[12,  1400] loss: 0.049
[12,  1600] loss: 0.071
[12,  1800] loss: 0.058
result of epoch 11accuracy is : 95 %
[13,   200] loss: 0.059
[13,   400] loss: 0.061
[13,   600] loss: 0.064
[13,   800] loss: 0.050
[13,  1000] loss: 0.052
[13,  1200] loss: 0.070
[13,  1400] loss: 0.071
[13,  1600] loss: 0.055
[13,  1800] loss: 0.084
result of epoch 12accuracy is : 95 %
[14,   200] loss: 0.065
[14,   400] loss: 0.048
[14,   600] loss: 0.064
[14,   800] loss: 0.067
[14,  1000] loss: 0.067
[14,  1200] loss: 0.057
[14,  1400] loss: 0.062
[14,  1600] loss: 0.061
[14,  1800] loss: 0.053
result of epoch 13accuracy is : 96 %
[15,   200] loss: 0.058
[15,   400] loss: 0.054
[15,   600] loss: 0.043
[15,   800] loss: 0.059
[15,  1000] loss: 0.057
[15,  1200] loss: 0.068
[15,  1400] loss: 0.059
[15,  1600] loss: 0.071
[15,  1800] loss: 0.063
result of epoch 14accurac

[44,  1200] loss: 0.021
[44,  1400] loss: 0.022
[44,  1600] loss: 0.034
[44,  1800] loss: 0.027
result of epoch 43accuracy is : 95 %
[45,   200] loss: 0.025
[45,   400] loss: 0.020
[45,   600] loss: 0.023
[45,   800] loss: 0.028
[45,  1000] loss: 0.026
[45,  1200] loss: 0.026
[45,  1400] loss: 0.023
[45,  1600] loss: 0.021
[45,  1800] loss: 0.026
result of epoch 44accuracy is : 96 %
[46,   200] loss: 0.022
[46,   400] loss: 0.017
[46,   600] loss: 0.024
[46,   800] loss: 0.025
[46,  1000] loss: 0.018
[46,  1200] loss: 0.028
[46,  1400] loss: 0.032
[46,  1600] loss: 0.024
[46,  1800] loss: 0.031
result of epoch 45accuracy is : 95 %
[47,   200] loss: 0.019
[47,   400] loss: 0.023
[47,   600] loss: 0.021
[47,   800] loss: 0.028
[47,  1000] loss: 0.023
[47,  1200] loss: 0.022
[47,  1400] loss: 0.023
[47,  1600] loss: 0.022
[47,  1800] loss: 0.034
result of epoch 46accuracy is : 96 %
[48,   200] loss: 0.027
[48,   400] loss: 0.023
[48,   600] loss: 0.021
[48,   800] loss: 0.018
[48,  1000] 

result of epoch 21accuracy is : 95 %
[23,   200] loss: 0.029
[23,   400] loss: 0.025
[23,   600] loss: 0.019
[23,   800] loss: 0.024
[23,  1000] loss: 0.024
[23,  1200] loss: 0.022
[23,  1400] loss: 0.022
[23,  1600] loss: 0.027
[23,  1800] loss: 0.024
result of epoch 22accuracy is : 95 %
[24,   200] loss: 0.025
[24,   400] loss: 0.018
[24,   600] loss: 0.025
[24,   800] loss: 0.024
[24,  1000] loss: 0.022
[24,  1200] loss: 0.018
[24,  1400] loss: 0.021
[24,  1600] loss: 0.019
[24,  1800] loss: 0.023
result of epoch 23accuracy is : 95 %
[25,   200] loss: 0.022
[25,   400] loss: 0.017
[25,   600] loss: 0.019
[25,   800] loss: 0.017
[25,  1000] loss: 0.028
[25,  1200] loss: 0.019
[25,  1400] loss: 0.029
[25,  1600] loss: 0.020
[25,  1800] loss: 0.021
result of epoch 24accuracy is : 95 %
[26,   200] loss: 0.023
[26,   400] loss: 0.015
[26,   600] loss: 0.016
[26,   800] loss: 0.020
[26,  1000] loss: 0.016
[26,  1200] loss: 0.023
[26,  1400] loss: 0.023
[26,  1600] loss: 0.011
[26,  1800] 

In [59]:
len(han_x_test),len(x_test_sent_vecs)

(2364, 10)

In [60]:
import torch.nn as nn
import torch.nn.functional as F
net = Net(pretrained, len(pretrained))
cur_max_acc = 0.0
print(net)
from torch import optim
criterion = nn.CrossEntropyLoss() # 交叉熵损失函数
optimizer = optim.Adagrad(net.parameters(), lr=0.01)#, momentum=0.9)
result_set = {}
for key in sub_set:
    print(key)
    res = []
    with torch.no_grad():
        correct, total = 0, 0
        testset = [(han_x_test[i],np.array(x_test_sent_vecs[key][i])) for i in range(len(han_x_test))]
        testloader = torch.utils.data.DataLoader(
                            testset, 
                            batch_size=4,
                            shuffle=False, 
                            num_workers=2)
        for data in testloader:
            # 梯度清零
            optimizer.zero_grad()
            # forward + backward 
            (inputs, sentis) = data
            va = [[word2ind[key]]]
            va = torch.from_numpy(np.array(va))
            net = torch.load('attention++/net' + key + '.pkl')
            outputs,alpha = net(inputs, sentis, va)
            _, predicted = torch.max(outputs, 1)
            res = res + [map_[int(predicted[i])] for i in range(len(predicted))]
    result_set[key] = res


Net(
  (bilstm): LSTM(100, 100, bidirectional=True)
  (fch): Linear(in_features=200, out_features=100, bias=True)
  (fcv): Linear(in_features=200, out_features=100, bias=True)
  (fcw): Linear(in_features=200, out_features=1, bias=True)
  (fcwp): Linear(in_features=200, out_features=50, bias=True)
  (fcwh): Linear(in_features=200, out_features=50, bias=True)
  (fcwsen): Linear(in_features=200, out_features=50, bias=True)
  (fc1): Linear(in_features=3750, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=10, bias=True)
  (fcnn): Linear(in_features=50, out_features=10, bias=True)
  (final): Linear(in_features=10, out_features=4, bias=True)
  (cnn): Conv2d(1, 50, kernel_size=(3, 100), stride=(1, 1))
  (pooling): MaxPool2d(kernel_size=(1, 48), stride=(1, 48), padding=0, dilation=1, ceil_mode=False)
  (embeddings): Embedding(19307, 100)
)
价格


/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:90: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


配置
操控
舒适性
油耗
动力
内饰
安全性
空间
外观


In [61]:
'''
data = pd.read_csv('../../data/df_sen_sub/train.csv')
uni_sub = data['subject'].unique()
for key in uni_sub:
    print(key + str(len(data[data['subject'] == key])))
    ttt = data[data['subject'] == key]
    print(len(ttt[ttt['sentiment_value'] == 0]))
    print(len(ttt[ttt['sentiment_value'] == -1]))
    print(len(ttt[ttt['sentiment_value'] == 1]))
'''

"\ndata = pd.read_csv('../../data/df_sen_sub/train.csv')\nuni_sub = data['subject'].unique()\nfor key in uni_sub:\n    print(key + str(len(data[data['subject'] == key])))\n    ttt = data[data['subject'] == key]\n    print(len(ttt[ttt['sentiment_value'] == 0]))\n    print(len(ttt[ttt['sentiment_value'] == -1]))\n    print(len(ttt[ttt['sentiment_value'] == 1]))\n"

In [62]:
real_pred = {}
content_ids = []
subjects = []
sentiment_values = []
sentiment_words = []
contents = []
#content_id,subject,sentiment_value,sentiment_word
for i in range(len(test_data)):
    content_id = test_data.loc[i, 'content_id']
    flag = 0
    for key in result_set:
        if result_set[key][i] != '2':
            flag = 1
            content_ids.append(str(content_id))
            subjects.append(key)
            sentiment_values.append(int(result_set[key][i]))
            sentiment_words.append("")
            contents.append(str(test_data.loc[i, 'content']))
    if flag == 0:
        content_ids.append(str(content_id))
        subjects.append('未知')
        sentiment_values.append(0)
        sentiment_words.append("")   
        contents.append(str(test_data.loc[i, 'content']))
real_pred = {"content_id":content_ids, "subject":subjects, "sentiment_value":sentiment_values, "sentiment_word":sentiment_words}
real_df = pd.DataFrame(real_pred)
real_df.to_csv('attention++/final_result++.csv', encoding = "utf-8", index = False)

In [63]:
len(real_df)

2599

In [53]:
real_pred['content'] = contents

dtt = pd.DataFrame(real_pred)
dtt.to_csv('attention++/copy_see_final_result++.csv', encoding = "utf-8", index = False)